## Setting up

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException,ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random

options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')  # Disable automation flags
options.add_experimental_option('excludeSwitches', ['enable-logging'])
# options.add_experimental_option(
#         "prefs", {
#             # block image loading
#             "profile.managed_default_content_settings.images": 2,
#         }
#     )
# options.add_argument('--headless')

service = Service(ChromeDriverManager().install())

# driver = webdriver.Chrome(service=service, options=options)
# wait = WebDriverWait(driver, 10)
# driver.get("https://alonhadat.com.vn/can-ban-nha-da-nang-t3/trang-200.htm")


## Supporting functions

In [8]:
data_list = []

In [9]:

def human_like_delay(min_time=1, max_time=3):
    time.sleep(random.uniform(min_time, max_time))

def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    human_like_delay(1, 3)


## Scrape page function

In [10]:
def scrape_page(url):
    try:
        driver.get(url)
        scroll_to_bottom()
        human_like_delay(2, 4)
        wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="content-item"]')))
        cards = driver.find_elements(By.XPATH, '//div[@class="content-item"]')

        for i,card in enumerate(cards):
            driver.execute_script("arguments[0].scrollIntoView();", cards[i])
            human_like_delay(1, 2)
            data_dict = {}
            try:
                area = card.find_element(By.XPATH, './/div[@class="square-direct"]//div[@class="ct_dt"]').text[10:].strip()
            except:
                area = None

            try:
                road_width = card.find_element(By.XPATH, './/div[@class="characteristics"]//span[@class="road-width"]').text.strip()
            except:
                road_width = None

            try:
                floors = card.find_element(By.XPATH, './/div[@class="characteristics"]//span[@class="floors"]').text.strip()
            except:
                floors = None
            try:
                bedroom = card.find_element(By.XPATH, './/div[@class="characteristics"]//span[@class="bedroom"]').text.strip()
            except:
                bedroom = None
            try:
                garage = card.find_element(By.XPATH, './/div[@class="characteristics"]//span[@class="parking"]').text.strip()
            except:
                garage = 0
            try:
                price = card.find_element(By.XPATH, './/div[@class="price-dis"]//div[@class="ct_price"]').text[3:].strip()
            except:
                price = None
            try:
                loc = ""
                parent_div = card.find_element(By.XPATH, './/div[@class="price-dis"]//div[@class="ct_dis"]')
                atags = parent_div.find_elements(By.TAG_NAME, 'a')
                for atag in atags:
                    loc += atag.text.strip()+", "
                loc = loc[:-1]
            except:
                loc = None

            data_dict['price'] = price
            data_dict['area'] = area
            data_dict['road_width'] = road_width
            data_dict['floors'] = floors
            data_dict['bedroom'] = bedroom
            data_dict['loc'] = loc
            data_dict['garage'] = garage
            
            data_list.append(data_dict)
            human_like_delay(1, 4)
            print(len(data_list))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException) as e:
        print(f"Error: {str(e)}")



In [11]:
data_list = []

## Checkpoint

In [13]:
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
index = 0
try:
    for i in range(353,500):
        try:
            url = f'https://alonhadat.com.vn/can-ban-nha-da-nang-t3/trang-{i}.htm'
            scrape_page(url)
            index = i
        except:
            break
except Exception:
    print("An error occurred while scraping the page.")
finally:
    print(f"Scraping until {index}")
    print(data_list)
    df = pd.DataFrame(data_list)
    df.to_csv('scraped_data_alonhadat_onlyfront.csv',encoding="utf-8", index=False)
    print("Saved successfully")
# driver.quit()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
